In [7]:
def warn(*args, **kwargs):
    pass

import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [ ]:
# OLLAMA_HOST=127.0.0.1:11500 ollama pull llama3
# ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIGIBF4ybPVTD9MroHmY6kKkkWP0sLdW3JfmY2cQAkX9S
# nous-hermes-2-mistral-7b-dpo

In [ ]:
# Import Libraries
import os
# os.environ["OLLAMA_HOST"] = "127.0.0.1:11500"
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.llms import ollama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

import pickle

In [ ]:
# Load and Process Documents

directory = "../docs/"
all_docs = []

for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        file_path = os.path.join(directory, filename)
        loader = PyPDFLoader(file_path)
        documents = loader.load()
        all_docs.extend(documents)


In [10]:
# Chunking

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(all_docs)


In [12]:
# Create Embeddings and Store
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
# import torch
# torch.cuda.is_available()

In [16]:
# Store embeddings in ChromaDB

if os.path.exists('chroma_db'):
    # Load
    vectorstore = Chroma(embedding_function=embeddings, persist_directory="chroma_db")
else:
    # If not found -> throw error -> create new
    vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory="chroma_db")

In [17]:
# Set Up the LLM

llm = ollama.Ollama(model="hermes3:8b")

In [18]:
# Create a Prompt

system_prompt = """
        You are an AI lawyer specializing in Indian law.
        Your role is to provide clear, concise, and accurate legal advice based solely on the information from the provided documents and prior conversations with the user.
        You must always respond as a legal expert and avoid disclaiming your expertise.
        If an answer is unknown, simply state that and refrain from speculation.
        Cite relevant law sections, acts, or provisions in your response.
        Note: The developer has provided the legal documents, not the user.

        Previous conversations:
        {history}

        Document context:
        {context}
    """

qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )